In [1]:
import cv2
import numpy as np
from numpy import loadtxt
import sys

#from pylab import *
#%pylab inline
#pylab.rcParams['figure.figsize'] = (50, 10)
#np.set_printoptions(precision = 17)

#%matplotlib inline
#import matplotlib.pyplot as plt
import scipy

In [10]:
import PIL.Image
from cStringIO import StringIO
import IPython.display

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [2]:
# caffe layers
caffe_root = '/users/vijay.kumar/caffe/'
sys.path.insert(0, caffe_root + 'python')

import caffe
from caffe import layers as L

# enable gpu
caffe.set_mode_gpu()

In [35]:
# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': (1,3,224,224)})
transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
#transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR # not required..

In [31]:
proto_file = '/users/vijay.kumar/code/person_recognition/models/new/vgg_face_caffe/VGG_FACE_deploy.prototxt'
weights = '/users/vijay.kumar/code/person_recognition/models/new/vgg_face_caffe/VGG_FACE.caffemodel'
net = caffe.Net(proto_file, weights, caffe.TEST)
net.forward()
avgImg = [129.1863,104.7624,93.5940]

In [41]:
album_gt_path = 'data/G-album/GallagherDatasetGT.txt'
image_dir = 'data/G-album/data/'
count = 0
num_examples = sum(1 for line in open(album_gt_path))
features = np.zeros((num_examples, 4096))
with open(album_gt_path) as f:
    for line in f:        
        line_split = line.strip().split('\t')
        img_name = line_split[0]
        label = line_split[5]
        image = cv2.imread(image_dir + img_name)     ## NOTE USING OPENCV TO READ.. # BGR Format
        x1 = int(line_split[1])
        y1 = int(line_split[2])
        x2 = int(line_split[3])
        y2 = int(line_split[4])
        
        new_x1 = x1 - 0.5*abs(x1-x2)
        if new_x1 < 0: 
            new_x1 = 0
        
        new_x2 = x2 + 0.5*abs(x1-x2)
        if new_x2 > image.shape[1]:
            new_x2 = image.shape[1]
        
        new_y1 = y1 - 0.5*abs(x1-x2)
        if new_y1 < 0:
            new_y1 = 0
        f_w = new_x2 - new_x1
        f_h = f_w
        
        face = image[new_y1:new_y1+f_h, new_x1:new_x1+f_w,:]
        face[0] = face[0] - avgImg[2]
        face[1] = face[1] - avgImg[1]
        face[2] = face[2] - avgImg[0]
    
        #showarray(face)
        transformed_image = transformer.preprocess('data', face)                
        net.blobs['data'].data[0, ...] = transformed_image
        net.forward(start='conv1_1')
        features[count] = net.blobs['fc7'].data
        count = count + 1        
        
        if count%100 == 0:
            print count
np.savez('/home/vijay.kumar/semi-supervised/galbum_feas',features)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


931
931
931
931
931
931
931
931
931
